In [25]:
import os, json

import unstructured_client
from unstructured_client.models import operations, shared

client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)


In [ ]:
import os, json
import unstructured_client
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError
from dotenv import load_dotenv
load_dotenv()

def get_elements_from_file(filename):
    with open(filename, "rb") as f:          # pass bytes, not a file handle
        file_bytes = f.read()
            # print first 10 bytes

    req = operations.PartitionRequest(
        partition_parameters=shared.PartitionParameters(
            strategy=shared.Strategy.FAST,
            files=shared.Files(
                content=file_bytes,
                file_name=filename,
                
            ),
            
            languages=["heb", "eng"],
        )
    )

    client = unstructured_client.UnstructuredClient(
        api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
    )

    try:  
        res = client.general.partition(request=req)
        elements = res.elements                          # list of Element objects
        
        return elements
    except SDKError as e:                                # friendlier error handling
        print(f"{e.status_code}: {e.message}")


In [2]:
os.getenv("UNSTRUCTURED_API_KEY")

'2CuIov3dF0K9vnm1zu39yJa7QdYqNe'

In [60]:
from pathlib import Path
from typing import List

from pathlib import Path
from typing import List, Tuple

def list_all_files(
    start: str | Path,
    exts: Tuple[str, ...] = (".pdf", ".doc", ".docx"),
) -> List[str]:
    """
    Return a list of full (absolute) paths for every *PDF, DOC, or DOCX* file
    inside *start* and all its sub-directories.

    Parameters
    ----------
    start : str | Path
        The root folder to walk. Tilde (~) is expanded and the path is
        resolved so the result is always absolute.
    exts : tuple[str, ...], optional
        File-name suffixes to include (case-insensitive). Default picks
        '.pdf', '.doc', and '.docx'.

    Raises
    ------
    FileNotFoundError
        If *start* does not exist or is not a directory.
    """
    root = Path(start).expanduser().resolve()

    if not root.is_dir():
        raise FileNotFoundError(f"{root} is not an existing directory")

    # rglob('*') walks recursively; filter by suffix (case-insensitive)
    return [
        str(p)
        for p in root.rglob("*")
        if p.is_file() and p.suffix.lower() in exts
    ]


pathes=list_all_files("docs")


In [62]:
for path in pathes:
    print(path)
    

/Users/matansharon/python/AI/unsructered/docs/ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf
/Users/matansharon/python/AI/unsructered/docs/GaLore- Memory-Efficient LLM Training by Gradient Low-Rank Projection.pdf
/Users/matansharon/python/AI/unsructered/docs/2403.06634.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manifolds - Rev B_scanned for TF.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/Risk Management Plan - Bonded Manifolds.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manufolds - Rev A.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMF - bonded manifolds - Rev C.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMF- Bonded Manifolds -Rev G_scanned for TF.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP -

In [63]:
unstructerd_files=[]
for i in range(len(pathes)):
    el=get_elements_from_file(pathes[i])
    unstructerd_files.append(el)


INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1

23

In [67]:
import pandas as pd
cols=['type','element_id','text','metadata','filename']
df=pd.DataFrame(columns=cols)
df

,type,element_id,text,metadata,filename


In [68]:
rows_list = []
for element in unstructerd_files:
    for row in element:
        rows_list.append({
            'type': row['type'],
            'filename': row['metadata']['filename'],
            'element_id': row['element_id'],
            'text': row['text'],
            'metadata': row['metadata']
        })
unstructerd_df = pd.DataFrame(rows_list, columns=cols)




In [77]:
len(unstructerd_df)

3221

In [16]:
#iterate over the rows of the DataFrame and check if there is any duplicate text
for index, row in df.iterrows():
    if df['text'].duplicated().any():
        print(f"Duplicate text found in row {index}: {row['text']}")
    else:
        print(f"No duplicate text found in row {index}: {row['text']}")


Duplicate text found in row 0: 3-01-21  SOP 4  Appendix Appendix status - released Revision  5 Valid from ‏25/12/2016 Product/s name Bonded Manifolds Document status Under Edit Author - Title Revision B Risk Management Plan Release Date - Document# Doc-075384 Page 8 of 8
Duplicate text found in row 1: Company: Elcam Medical
Duplicate text found in row 2: Product Description:
Duplicate text found in row 3: The Bonded Manifolds family of products contains products that are alternative for Manifold-Monoblocks. They are 2,3 and 4 gang UV bonded manifold, LB and closed stopcocks.  
Duplicate text found in row 4: This Risk management plan covers all risk management activities required for bonded manifolds development projects as well as their design change projects.
Duplicate text found in row 5: Their lifecycle stages of such projects are divided to:
Duplicate text found in row 6: project initiation
Duplicate text found in row 7: Design Planning
Duplicate text found in row 8: Design Input
D

In [37]:
import pandas as pd
import unstructured_client
client=unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)

# # Example document path - adjust as needed
# document_path = "path/to/your/document.pdf"  # Change this to your 
# document path

# # Extract elements from document
# elements = partition(document_path)

# # Convert elements to a pandas DataFrame
# rows = []
# for element in elements:
#     # Extract common properties
#     element_data = {
#         'type': element.category,
#         'element_id': element.id,
#         'text': element.text
#     }

#     # Add metadata as separate columns
#     if hasattr(element, 'metadata'):
#         metadata = element.metadata.to_dict() if
# hasattr(element.metadata, 'to_dict') else element.metadata

#         # Add common metadata fields 
#         for field in ['filename', 'file_directory', 'last_modified',
# 'filetype', 'page_number', 'parent_id', 'languages']:
#             element_data[field] = metadata.get(field, None)

#     rows.append(element_data)

# # Create DataFrame
# df = pd.DataFrame(rows)

# # Example usage:
# print(f"Created DataFrame with {len(df)} elements")
# print(df.head())

# # Optional: Filter to specific element types
# narrative_text = df[df['type'] == 'NarrativeText']
# tables = df[df['type'] == 'Table']

Help on method partition in module unstructured_client.general:

partition(*, request: Union[unstructured_client.models.operations.partition.PartitionRequest, unstructured_client.models.operations.partition.PartitionRequestTypedDict], retries: OptionalNullable[unstructured_client.utils.retries.RetryConfig] = Unset(), server_url: Optional[str] = None, timeout_ms: Optional[int] = None, accept_header_override: Optional[unstructured_client.general.PartitionAcceptEnum] = None, http_headers: Optional[Mapping[str, str]] = None) -> unstructured_client.models.operations.partition.PartitionResponse method of unstructured_client.general.General instance
    Summary

    Description

    :param request: The request object to send.
    :param retries: Override the default retry configuration for this method
    :param server_url: Override the default server URL for this method
    :param timeout_ms: Override the default request timeout configuration for this method in milliseconds
    :param accept